In [5]:

import os
import pandas as pd
from pathlib import Path
import tensorflow.keras

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint ,ReduceLROnPlateau, Callback, LearningRateScheduler
import sys
import datetime
import math
import numpy as np
from tensorflow.keras import backend as K

from tensorflow.keras.models import load_model
import logging
#from sklearn.metrics import roc_auc_score

%run dictionary.ipynb
%run dataloader.ipynb
%run metrics.ipynb
%run unet_costum.ipynb


'''
  Segmentation Pulmonary Aorta
'''


def train(params) :
    
    
    # Allow GPU growth
    '''
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
    config.log_device_placement = True  # to log device placement (on which device the operation ran)
    sess = tf.compat.v1.Session(config=config)
    tf.compat.v1.keras.backend.set_session(sess)  # set this TensorFlow session as the default session for Keras
    )
    
        
    
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
    '''
    #strategy = tf.distribute.MirroredStrategy()


    # Open a strategy scope.for using single/multiple GPUs)
    #with strategy.scope():
        
    #Dataset
    x_train, y_train, x_val, y_val = data_dict(params['dataset'] + '.csv', params['val_size'], params['random_state'])



    #initialize generators 
    training_generator = DataGenerator(x_train, y_train, params["epochs"], params['batch_size'],
                                   params["dim"], params['n_channels'],
                                   params["shuffle"],
                                   params['num_output_nodes'], params['augmentation'],params['crop_pad'])

    validation_generator = DataGenerator(x_val, y_val, params["epochs"],
                                         params['batch_size'],
                                         params["dim"], params['n_channels'],
                                         False, params['num_output_nodes'], False,params['crop_pad'])

    #model

    model =  unet_costum(input_shape = params['dim'], dropout=params['dropout'], output_activation = "sigmoid")


    #optimizer
    if params["optimizer"] == "sgd" :
        optimizer = tf.keras.optimizers.SGD(learning_rate=params['learning_rate'],
                                            momentum=0.9, nesterov=params['nesterov'], name="SGD")
    elif params["optimizer"] == "adam" :
        optimizer = tf.keras.optimizers.Adam(lr=params['learning_rate'])



    #METRICS = ["accuracy"]
    #METRICS = [f1_sk,jaccard_sk]
    METRICS = [f1,jaccard]
    
    # Compile the model
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                      optimizer=optimizer,
                      metrics=METRICS)

    def lr_step_decay(epoch) :
        # https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
        drop_rate = 0.5
        epochs_drop = 5
        return params['learning_rate'] * math.pow(drop_rate, math.floor(epoch / epochs_drop))

    def lr_steady(epoch) :
        return params['learning_rate']

    if params["lr_decay"] == "plateau" :
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.4, patience=2, verbose=1,
                                      mode='min', min_delta=0, cooldown=0, min_lr=0)
    elif params["lr_decay"] == "steps" :
        reduce_lr = LearningRateScheduler(lr_step_decay, verbose=1)

    elif params["lr_decay"] == False :
        reduce_lr = LearningRateScheduler(lr_steady, verbose=1)

    else :
        print("Learning rate decay unknown!")

    class LearningRateLogger(tf.keras.callbacks.Callback) :
        def on_epoch_end(self, epoch, logs) :
            logs['z_learningrate'] = K.eval(self.model.optimizer.lr)  # replace it with your metrics


    # Checkpoints
    if params['history'] == True :
        my_checkpoints = [
            ModelCheckpoint('./models/'+ params['comment'] + ".hdf5", monitor='val_loss', verbose=1,
                            save_best_only=True, mode='min', period=1),
            LearningRateLogger(),
            tf.keras.callbacks.CSVLogger('./models/' + str(model_name) + '.log', separator=',',
                                         append=False),
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode="min", patience=5, restore_best_weights=True),
            reduce_lr
            ]
    else :
        my_checkpoints = None
    
    workers = 3

    # Train model on dataset
    model.fit(x = training_generator,
             steps_per_epoch=len(x_train) // params['batch_size'],
              validation_data = validation_generator,
                  validation_steps=len(x_val) // params['batch_size'],
                  epochs=params['epochs'],
                  callbacks=my_checkpoints,
                    workers=workers
                       )




params = {'dim' : (256, 256, 4),  # (depth,length,width)
      'batch_size' : 3,
      'num_output_nodes' : 1, 
      'n_channels' : 1,
      'epochs' : 100,
      'shuffle' : True,
      'crop_pad': False,
      'learning_rate' : 0.0001,
      'optimizer' : "adam",
      'momentum' : 0,
      'lr_decay' : "plateau",
      'nesterov' : False,
      'reg_factor' : 0.001,
      'model_type' : "unet_costum",
      'dropout' : 0,
      'history' : False,
      'val_size' : 0.3,
      'test_size' : 0,
      'random_state' : 123,  # identical sample_size required
      'dataset' : "mapping",
      'augmentation' : 0, # float between 0 and 1, define the probablilite of the transf funktion being called
     'comment' : "baseline_more_data" #comment 
         }
train(params)

Total samples:  25
17
val_split:  7
Samples training set:  17
Samples validation set:  7
Epoch 1/100
5/5 [==============================] - 14s 3s/step - loss: 0.8242 - f1: 0.2707 - jaccard: 0.1568 - val_loss: 0.6953 - val_f1: 0.2339 - val_jaccard: 0.1362
Epoch 2/100
5/5 [==============================] - 11s 2s/step - loss: 0.7988 - f1: 0.2878 - jaccard: 0.1694 - val_loss: 0.7010 - val_f1: 0.2412 - val_jaccard: 0.1406
Epoch 3/100
5/5 [==============================] - 11s 2s/step - loss: 0.7748 - f1: 0.3007 - jaccard: 0.1771 - val_loss: 0.7031 - val_f1: 0.2424 - val_jaccard: 0.1412
Epoch 4/100
5/5 [==============================] - 11s 2s/step - loss: 0.7558 - f1: 0.3146 - jaccard: 0.1874 - val_loss: 0.6999 - val_f1: 0.2467 - val_jaccard: 0.1443
Epoch 5/100
5/5 [==============================] - 11s 2s/step - loss: 0.7374 - f1: 0.3337 - jaccard: 0.2009 - val_loss: 0.6928 - val_f1: 0.2464 - val_jaccard: 0.1447
Epoch 6/100
5/5 [==============================] - 11s 2s/step - loss: 0.723

KeyboardInterrupt: 